In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/abalone-dataset/abalone.csv
/kaggle/input/playground-series-s4e4/sample_submission.csv
/kaggle/input/playground-series-s4e4/train.csv
/kaggle/input/playground-series-s4e4/test.csv


In [2]:
import pandas as pd, numpy as np

In [3]:
dataTrain = pd.read_csv("/kaggle/input/playground-series-s4e4/train.csv")
dataTest = pd.read_csv("/kaggle/input/playground-series-s4e4/test.csv")

In [4]:
dataTrain = dataTrain.rename(columns={'Whole weight.1': 'Shucked weight', 'Whole weight.2': 'Viscera weight'})
dataTest = dataTest.rename(columns={'Whole weight.1': 'Shucked weight', 'Whole weight.2': 'Viscera weight'})

In [5]:
dataTest.head()

,id,Sex,Length,Diameter,Height,Whole weight,Shucked weight,Viscera weight,Shell weight
0,90615,M,0.645,0.475,0.155,1.2380,0.6185,0.3125,0.3005
1,90616,M,0.580,0.460,0.160,0.9830,0.4785,0.2195,0.2750
2,90617,M,0.560,0.420,0.140,0.8395,0.3525,0.1845,0.2405
3,90618,M,0.570,0.490,0.145,0.8740,0.3525,0.1865,0.2350
4,90619,I,0.415,0.325,0.110,0.3580,0.1575,0.0670,0.1050


In [6]:
dataTrain.head()

,id,Sex,Length,Diameter,Height,Whole weight,Shucked weight,Viscera weight,Shell weight,Rings
0,0,F,0.550,0.430,0.150,0.7715,0.3285,0.1465,0.2400,11
1,1,F,0.630,0.490,0.145,1.1300,0.4580,0.2765,0.3200,11
2,2,I,0.160,0.110,0.025,0.0210,0.0055,0.0030,0.0050,6
3,3,M,0.595,0.475,0.150,0.9145,0.3755,0.2055,0.2500,10
4,4,I,0.555,0.425,0.130,0.7820,0.3695,0.1600,0.1975,9


In [7]:
dataTrain.describe()

,id,Length,Diameter,Height,Whole weight,Shucked weight,Viscera weight,Shell weight,Rings
count,90615.000000,90615.000000,90615.000000,90615.000000,90615.000000,90615.000000,90615.000000,90615.000000,90615.000000
mean,45307.000000,0.517098,0.401679,0.135464,0.789035,0.340778,0.169422,0.225898,9.696794
std,26158.441658,0.118217,0.098026,0.038008,0.457671,0.204428,0.100909,0.130203,3.176221
min,0.000000,0.075000,0.055000,0.000000,0.002000,0.001000,0.000500,0.001500,1.000000
25%,22653.500000,0.445000,0.345000,0.110000,0.419000,0.177500,0.086500,0.120000,8.000000
50%,45307.000000,0.545000,0.425000,0.140000,0.799500,0.330000,0.166000,0.225000,9.000000
75%,67960.500000,0.600000,0.470000,0.160000,1.067500,0.463000,0.232500,0.305000,11.000000
max,90614.000000,0.815000,0.650000,1.130000,2.825500,1.488000,0.760000,1.005000,29.000000


In [8]:
dataTrain.isnull().sum()

id                0
Sex               0
Length            0
Diameter          0
Height            0
Whole weight      0
Shucked weight    0
Viscera weight    0
Shell weight      0
Rings             0
dtype: int64

In [9]:
dataTrain.duplicated().sum()

0

In [10]:
categoricalData = dataTrain.select_dtypes("object")

In [11]:
categoricalData.nunique()

Sex    3
dtype: int64

In [12]:
dataTrain_encoded = pd.get_dummies(dataTrain, columns=['Sex'], drop_first=True)


In [13]:
dataTest.head()

,id,Sex,Length,Diameter,Height,Whole weight,Shucked weight,Viscera weight,Shell weight
0,90615,M,0.645,0.475,0.155,1.2380,0.6185,0.3125,0.3005
1,90616,M,0.580,0.460,0.160,0.9830,0.4785,0.2195,0.2750
2,90617,M,0.560,0.420,0.140,0.8395,0.3525,0.1845,0.2405
3,90618,M,0.570,0.490,0.145,0.8740,0.3525,0.1865,0.2350
4,90619,I,0.415,0.325,0.110,0.3580,0.1575,0.0670,0.1050


In [14]:
og = pd.read_csv("/kaggle/input/abalone-dataset/abalone.csv")

In [15]:
dataTest_encoded = pd.get_dummies(dataTest, columns=['Sex'], drop_first=True)
og = pd.get_dummies(og, columns=['Sex'], drop_first=True)

In [16]:
dataTrain_encoded.head()

,id,Length,Diameter,Height,Whole weight,Shucked weight,Viscera weight,Shell weight,Rings,Sex_I,Sex_M
0,0,0.550,0.430,0.150,0.7715,0.3285,0.1465,0.2400,11,False,False
1,1,0.630,0.490,0.145,1.1300,0.4580,0.2765,0.3200,11,False,False
2,2,0.160,0.110,0.025,0.0210,0.0055,0.0030,0.0050,6,True,False
3,3,0.595,0.475,0.150,0.9145,0.3755,0.2055,0.2500,10,False,True
4,4,0.555,0.425,0.130,0.7820,0.3695,0.1600,0.1975,9,True,False


In [17]:
dataTest_encoded.drop(columns = 'id', inplace = True)

In [18]:
dataTest_encoded.head()

,Length,Diameter,Height,Whole weight,Shucked weight,Viscera weight,Shell weight,Sex_I,Sex_M
0,0.645,0.475,0.155,1.2380,0.6185,0.3125,0.3005,False,True
1,0.580,0.460,0.160,0.9830,0.4785,0.2195,0.2750,False,True
2,0.560,0.420,0.140,0.8395,0.3525,0.1845,0.2405,False,True
3,0.570,0.490,0.145,0.8740,0.3525,0.1865,0.2350,False,True
4,0.415,0.325,0.110,0.3580,0.1575,0.0670,0.1050,True,False


In [19]:
og.head()

,Length,Diameter,Height,Whole weight,Shucked weight,Viscera weight,Shell weight,Rings,Sex_I,Sex_M
0,0.455,0.365,0.095,0.5140,0.2245,0.1010,0.150,15,False,True
1,0.350,0.265,0.090,0.2255,0.0995,0.0485,0.070,7,False,True
2,0.530,0.420,0.135,0.6770,0.2565,0.1415,0.210,9,False,False
3,0.440,0.365,0.125,0.5160,0.2155,0.1140,0.155,10,False,True
4,0.330,0.255,0.080,0.2050,0.0895,0.0395,0.055,7,True,False


In [20]:
dataTrain_encoded.drop(columns="id",inplace= True)

In [21]:
dataTrain_encoded["Sex_I"] = dataTrain_encoded["Sex_I"].astype(int)
dataTrain_encoded["Sex_M"] = dataTrain_encoded["Sex_M"].astype(int)

In [22]:
dataTest_encoded["Sex_I"] = dataTest_encoded["Sex_I"].astype(int)
dataTest_encoded["Sex_M"] = dataTest_encoded["Sex_M"].astype(int)

In [23]:
og["Sex_I"] = og["Sex_I"].astype(int)
og["Sex_M"] = og["Sex_M"].astype(int)

In [24]:
dataTrain_encoded.dtypes

Length            float64
Diameter          float64
Height            float64
Whole weight      float64
Shucked weight    float64
Viscera weight    float64
Shell weight      float64
Rings               int64
Sex_I               int64
Sex_M               int64
dtype: object

In [25]:
import xgboost as xgb
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.metrics import mean_squared_error

In [26]:
X = dataTrain_encoded.drop('Rings', axis=1)
y = dataTrain_encoded['Rings']

In [27]:
model = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=200, learning_rate=0.1, max_depth=7)

In [28]:
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# Perform cross-validation and store results
scores = cross_val_score(model, X, y, cv=kf, scoring='neg_mean_squared_error')
rmse_scores = (-scores)**0.5  # Convert MSE to RMSE

In [29]:
print("RMSE scores per fold:", rmse_scores)
print("Average RMSE:", rmse_scores.mean())

RMSE scores per fold: [1.86380489 1.83770594 1.85632818 1.85317424 1.82297325]
Average RMSE: 1.8467972997469164


In [30]:
model.fit(X, y)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.1, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=7, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=200, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [31]:
'''from sklearn.model_selection import GridSearchCV


# Define the model
model = xgb.XGBRegressor(objective='reg:squarederror')

# Create the parameter grid
param_grid = {
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.1, 0.2],
    'n_estimators': [100, 200, 300],
    'subsample': [0.7, 0.8, 0.9],
    'colsample_bytree': [0.7, 0.8, 0.9],
    'lambda': [1, 1.5],
    'alpha': [0, 0.5]
}

# Setup the GridSearch
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, scoring='neg_root_mean_squared_error', cv=5, verbose=1)

# Fit GridSearch to the data
grid_search.fit(X, y)

# Best parameters and RMSE
print("Best parameters found: ", grid_search.best_params_)
print("Best RMSE: ", -grid_search.best_score_)'''

'from sklearn.model_selection import GridSearchCV\n\n\n# Define the model\nmodel = xgb.XGBRegressor(objective=\'reg:squarederror\')\n\n# Create the parameter grid\nparam_grid = {\n    \'max_depth\': [3, 5, 7],\n    \'learning_rate\': [0.01, 0.1, 0.2],\n    \'n_estimators\': [100, 200, 300],\n    \'subsample\': [0.7, 0.8, 0.9],\n    \'colsample_bytree\': [0.7, 0.8, 0.9],\n    \'lambda\': [1, 1.5],\n    \'alpha\': [0, 0.5]\n}\n\n# Setup the GridSearch\ngrid_search = GridSearchCV(estimator=model, param_grid=param_grid, scoring=\'neg_root_mean_squared_error\', cv=5, verbose=1)\n\n# Fit GridSearch to the data\ngrid_search.fit(X, y)\n\n# Best parameters and RMSE\nprint("Best parameters found: ", grid_search.best_params_)\nprint("Best RMSE: ", -grid_search.best_score_)'

In [32]:

model = xgb.XGBRegressor(objective='reg:squarederror',alpha= 0, colsample_bytree= 0.8, reg_lambda= 1.5, learning_rate= 0.1, max_depth= 7, n_estimators= 200, subsample=0.9)                        

In [33]:
model.fit(X,y)

XGBRegressor(alpha=0, base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.8, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.1, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=7, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=200, n_jobs=None,
             num_parallel_tree=None, ...)

In [34]:
kf = KFold(n_splits=4, shuffle=True, random_state=42)

# Perform cross-validation and store results
scores = cross_val_score(model, X, y, cv=kf, scoring='neg_mean_squared_error')
rmse_scores = (-scores)**0.5  # Convert MSE to RMSE

In [35]:
print(rmse_scores)

[1.86212648 1.83990135 1.83841587 1.82824647]


In [36]:
final_data =pd.concat([dataTrain_encoded, og], ignore_index=True)

In [37]:
X = final_data.drop('Rings', axis=1)
y = final_data['Rings']

In [38]:
model.fit(X,y)

XGBRegressor(alpha=0, base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.8, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.1, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=7, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=200, n_jobs=None,
             num_parallel_tree=None, ...)

In [39]:
submit= pd.read_csv('/kaggle/input/playground-series-s4e4/sample_submission.csv')

In [40]:
submit.head()

,id,Rings
0,90615,10
1,90616,10
2,90617,10
3,90618,10
4,90619,10


In [41]:
dataTest_encoded.head()

,Length,Diameter,Height,Whole weight,Shucked weight,Viscera weight,Shell weight,Sex_I,Sex_M
0,0.645,0.475,0.155,1.2380,0.6185,0.3125,0.3005,0,1
1,0.580,0.460,0.160,0.9830,0.4785,0.2195,0.2750,0,1
2,0.560,0.420,0.140,0.8395,0.3525,0.1845,0.2405,0,1
3,0.570,0.490,0.145,0.8740,0.3525,0.1865,0.2350,0,1
4,0.415,0.325,0.110,0.3580,0.1575,0.0670,0.1050,1,0


In [42]:
pred_y = model.predict(dataTest_encoded)

In [43]:
submit.drop(columns='Rings',inplace=True)

In [44]:
submit['Rings']= pred_y

In [45]:
submit.head()

,id,Rings
0,90615,9.828862
1,90616,9.674077
2,90617,10.133894
3,90618,10.369923
4,90619,7.598033


In [46]:
submit.to_csv('submission.csv', index=False)